In [ ]:
# Standard libraries
import os
import random

# Third-party libraries
# Data Manipulation
import numpy as np
import pandas as pd

# Data Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning - General
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline as make_pipeline_imb

# Statistics and Model Evaluation
import statsmodels.api as sm
from scipy.stats import expon, reciprocal

# Visualisation for Machine Learning
from yellowbrick.cluster import KElbowVisualizer

# Neural Networks and Deep Learning
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier

# Hyperparameter Tuning
import kerastuner as kt

# Model Interpretability
import shap

# Local application/library specific imports
from collections import defaultdict
from kedro.pipeline import node

In [ ]:
%reload_kedro

In [ ]:
X_train = catalog.load("X_train_main")
y_train = catalog.load("y_train_main")
X_validate = catalog.load("X_validate_main")
y_validate = catalog.load("y_validate_main")
decision_tree_params = catalog.load("params:decision_tree")

In [ ]:
### Evaluation ##########################################################

def print_model_name(model_name):
    print(f"Evaluation Metrics: {model_name}")

def calculate_accuracy(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy}")
    return accuracy

def store_and_print_classification_report(y_test, y_pred):
    report = classification_report(y_test, y_pred)
    print(f"Classification Report:\n{report}")
    return report

def print_and_return_confusion_matrix(y_test, y_pred):
    matrix = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix:\n{matrix}")
    return matrix

def print_and_return_f1_score(y_test, y_pred):
    f1 = f1_score(y_test, y_pred)
    print(f"F1 Score: {f1}")
    return f1

def print_and_return_precision(y_test, y_pred):
    precision = precision_score(y_test, y_pred)
    print(f"Precision: {precision}")
    return precision

def print_and_return_recall(y_test, y_pred):
    recall = recall_score(y_test, y_pred)
    print(f"Recall: {recall}")
    return recall

def print_auc(model, X_test, y_test):
    """
    Prints the AUC for the given model and test data.
    """
    # Probabilities for the positive class
    probas = model.predict_proba(X_test)[:, 1]

    roc_auc = roc_auc_score(y_test, probas)

    print(f"AUC: {roc_auc}")
    return roc_auc

def print_auc_tf(model, X_test, y_test):
    """
    Prints the AUC for the given model and test data.
    """
    # Probabilities for the positive class
    probas = model.predict(X_test).ravel()

    roc_auc = roc_auc_score(y_test, probas)

    print(f"AUC: {roc_auc}")
    return roc_auc


def get_best_hyperparameters_decision_tree(grid_search): # Main Support
    best_params = grid_search.best_params_

    print(best_params)
    best_hyperparameters_df = pd.DataFrame(best_params, index=[0])

    return best_hyperparameters_df



In [ ]:
def train_decision_tree_with_grid_search(X_train, y_train, X_validate, y_validate, model_name): # Main Function   

    param_grid = {
        "decisiontreeclassifier__max_depth": [20],
        "decisiontreeclassifier__min_samples_split": range(10, 38),
        "decisiontreeclassifier__min_samples_leaf": range(33, 43),
        "decisiontreeclassifier__criterion": ["entropy"]
    }

    pipeline = make_pipeline_imb(StandardScaler(), SMOTE(random_state=42), 
                                 DecisionTreeClassifier(random_state=42))
    
    grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=10, scoring='accuracy', verbose=1)

    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_
    predictions = best_model.predict(X_validate)

    print_model_name(model_name)
    accuracy = calculate_accuracy(y_validate, predictions)
    auc = print_auc(best_model, X_validate, y_validate)
    f1 = print_and_return_f1_score(y_validate, predictions)
    precision = print_and_return_precision(y_validate, predictions)
    recall = print_and_return_recall(y_validate, predictions)

    

    # Get hyperparameter ranges
    best_params_df = get_best_hyperparameters_decision_tree(grid_search)
    confusion_matrix_values = print_and_return_confusion_matrix(y_validate, predictions)
#     f1 = print_and_return_f1_score(y_validate, predictions)
#     precision = print_and_return_precision(y_validate, predictions)
#     recall = print_and_return_recall(y_validate, predictions)

    tn, fp, fn, tp = confusion_matrix_values.ravel()


    # Store actual max depth 
    actual_max_depth = grid_search.best_estimator_.named_steps['decisiontreeclassifier']
    actual_max_depth = actual_max_depth.tree_.max_depth

    print('Decision Tree depth:', actual_max_depth)
    


    extracted_model = best_model.named_steps['decisiontreeclassifier']
    explainer = shap.Explainer(extracted_model, X_train)
    
    
    shap_values = explainer(X_train)
    print(type(shap_values))
    print(shap_values.shape)
    


    return {
        'shap_values': shap_values,
        'expected_value': explainer.expected_value
    }



In [ ]:
results = train_decision_tree_with_grid_search(X_train, y_train, X_validate, y_validate, "decision_tree")
shap_values = results['shap_values']
expected_value = results['expected_value']

In [ ]:
num_classes = shap_values.values.shape[-1]
for i in range(num_classes):
    class_shap_values = shap_values.values[..., i]
    print(f"Class {i} SHAP Values:")
    shap.summary_plot(class_shap_values, X_train)

In [ ]:
saved_shap_values = shap_values
saved_expected_value = expected_value

instance_index = 0  
class_index = 0  


In [ ]:
# Ensure that have numpy arrays for the SHAP values
if isinstance(saved_shap_values, shap.Explanation):
    saved_shap_values_array = saved_shap_values.values # Access values if its an explanation object 
else:
    saved_shap_values_array = saved_shap_values

# Pick the expected value for the class 
expected_value_for_class = saved_expected_value[class_index] if isinstance(saved_expected_value, np.ndarray) else saved_expected_value

# Pick the SHAP values for the specific instance and class
shap_values_for_instance_and_class = saved_shap_values_array[instance_index, :, class_index]

# plot
shap.decision_plot(expected_value_for_class, shap_values_for_instance_and_class, X_train.iloc[instance_index, :])
